In [58]:
import nltk
from nltk import word_tokenize


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import os

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# and, or, not - boolean operators
stop_words.remove('and')
stop_words.remove('or')
stop_words.remove('not')

In [2]:
def pickle_data(filename,file):
    filename = os.path.join('./pickle',filename)
    out = open(filename, 'wb')
    pickle.dump(file, out)
    out.close()
    
def load_pickle_data(file):
    file = os.path.join('./pickle',file)
    out = open(file, 'rb')
    index_dict = pickle.load(out)
    out.close()
    return index_dict

In [3]:
def remove_metadata(lines):
    for i in range(len(lines)):
        if lines[i] == '\n':
            start = i + 1
            break
    return lines[start:]



def process_text(lines, index_dict,doc_ID):
    lines = remove_metadata(lines)
    seperator = ' '
    file = seperator.join(lines)
    words = word_tokenize(file)
    words = process_words(words)
    
    for word in words:
        if word in index_dict.keys():
            if doc_ID not in index_dict[word]:
                index_dict[word].append(doc_ID)
        else:
            index_dict[word] = [doc_ID]
    
    return index_dict
            
                
def process_words(words):
    mod_words = []
    symbols = [ "'",'/','.','-','!','@','#','$','^','&','*','(',')','+']
    #words = [word.replace(sym,'') for word in words for sym in symbols if sym in word]
    removed_symbols = []
    for word in words :
        for sym in symbols:
            if sym in word:
                word = word.replace(sym,'')
        removed_symbols.append(word)
        
    words = removed_symbols
    del removed_symbols
        
    for word in words:
        word = word.lower()
        if word.isalnum():
            word = lemmatizer.lemmatize(word)
            if word not in stop_words and len(word) >= 2:
                mod_words.append(word)
    return mod_words
            
            
    

In [4]:
file_mapper = {}
root_dir = 'data'
doc_ID = 1
index_dict = {}
for fol in os.listdir(root_dir): # for each folder
    print(fol)
    path = os.path.join(root_dir,fol) 
    for file in os.listdir(path): # for each file in folder
        file_path = os.path.join(path, file)
        
        with open(file_path, 'r') as f:
            lines = f.readlines()
            file_mapper[doc_ID] = file_path
            index_dict = process_text(lines, index_dict,doc_ID)
            doc_ID += 1
pickle_data('index_dict.pkl',index_dict)
pickle_data('index_file_mapper.pkl',file_mapper)

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


###  Loading index dictionary

In [4]:
index_dict = load_pickle_data('index_dict.pkl')
file_mapper = load_pickle_data('index_file_mapper.pkl')
doc_ID = len(file_mapper)



In [17]:
def and_merge(post_list1, post_list2):

    
    ans = []
    ptr1 = 0 ; ptr2 = 0
    comparisons = 0
    while ptr1 < len(post_list1) and ptr2 < len(post_list2):
        
        if post_list1[ptr1] == post_list2[ptr2]:
            ans.append(post_list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif post_list1[ptr1] < post_list2[ptr2] :
            ptr1 += 1
        else :
            ptr2 += 1
        comparisons += 1
    return ans, comparisons

def or_merge(post_list1, post_list2):

    ans = []
    ptr1 = 0 ; ptr2 = 0
    comparisons = 0
    while ptr1 < len(post_list1) and ptr2 < len(post_list2):

        if post_list1[ptr1] == post_list2[ptr2]:
            ans.append(post_list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif post_list1[ptr1] < post_list2[ptr2] :
            ans.append(post_list1[ptr1])
            ptr1 += 1
        else :
            ans.append(post_list2[ptr2])
            ptr2 += 1
        comparisons += 1
        
    while(ptr1 < len(post_list1)):
        ans.append(post_list1[ptr1])
        ptr1 += 1

    while(ptr2 < len(post_list2)):
        ans.append(post_list2[ptr2])
        ptr2 += 1
        
    return ans, comparisons


def not_merge(post_list1,doc_ID):
    result = []
    for doc in range(1,doc_ID):
        if doc not in post_list1 :
            result.append(doc)

            
    return result

In [31]:
def retrieve_list(term,temp_results):
    ans = []
    if term in index_dict.keys():
        ans = index_dict[term]
    elif term in temp_results.keys():
        ans = temp_results[term]
    else :
        print('Postings list is not there for given term ')
    return ans


def processing_not_operator(query):
    temp_results = {}
    ind = 0
    i = len(query) - 1
    while i >= 0:
        if query[i] == 'not' :
            if query[i-1] == 'not':
                del query[i]
                del query[i-1]
                i -= 1
            else:
                term = query[i+1]
                post_list = retrieve_list(term,temp_results)
                key = 'result'+str(ind)
                ind += 1
                result = not_merge(post_list,doc_ID)
                temp_results[key] = result

                query[i] = key
                del query[i+1]
        i -= 1
    return query, temp_results, ind


In [32]:
def process_MIX(query, temp_results, ind):
    
    comparisons = 0
    while True:
        
        if 'and' in query:
            index = query.index('and')
            term1 = query[index-1]
            term2 = query[index+1]

            list1 = retrieve_list(term1, temp_results)
            list2 = retrieve_list(term2, temp_results)

            res,comp = and_merge(list1, list2)
            comparisons += comp
            key = 'result'+str(ind)
            ind += 1
            temp_results[key] = res
            query[index] = key
            del query[index-1]
            del query[index] # because of re arrangement
            print(query)
            
        else :
            
            query = [term for term in query if term != 'or']
            while len(query) > 1:
                term1 = query[0]
                term2 = query[1]
                list1 = retrieve_list(term1, temp_results)
                list2 = retrieve_list(term2, temp_results)
                
                res,comp = or_merge(list1, list2)
                comparisons += comp
                
                key = 'result'+str(ind)
                ind += 1
                temp_results[key] = res
                query[0] = key
                del query[1]
            break
    results = retrieve_list(query[0], temp_results)
    return results,comparisons
            


In [29]:
 

def all_and_operators(query):
    ans = True
    if 'or' in query :
        ans = False
    return ans
            
def find_minimum(lists):
    
    dummy = lists.copy()
    pos1 = lists.index(min(lists))
    del dummy[pos1]
    sec_min = min(dummy)
    pos2 = lists.index(sec_min)
    
    return pos1,pos2

def optimised_AND(query, temp_results, ind):
    results = []
    comparisons = 0
    if len(query) == 0 :
        print('Nothing to Process')
    else :
        print('OPTIMISING AND OPERATION')
        query = [term for term in query if term != 'and']
        posts_length = []
        for term in query:
            posts_length.append(len(retrieve_list(term,temp_results)))
        
        while True :
            
            if len(query) == 1:
                results = retrieve_list(query[0], temp_results)
                break
        
            else :
                print(posts_length)
                pos1,pos2 = find_minimum(posts_length)
                list1 = retrieve_list(query[pos1], temp_results)
                list2 = retrieve_list(query[pos2], temp_results)
                print('Performing AND between lists with length {} and {}'.format(len(list1),len(list2)))
                
                res,comp = and_merge(list1, list2)
                comparisons += comp
                key = 'result'+str(ind)
                ind += 1
                temp_results[key] = res
                posts_length[pos1] = len(res)
                query[pos1] = key
                
                del posts_length[pos2]
                del query[pos2]
    
    return results,comparisons
    
def process_query(query):
    
    temp_results = {}
    results = []
    query = process_words(query)
    print('***final query after preprocessing***')
    print(query)
    print('***Processing NOT Operator***')
    query, temp_results, ind = processing_not_operator(query)
    print(query)
    
    if all_and_operators(query) :
        results,comparisons = optimised_AND(query,temp_results, ind)
        
    else :
        print('**** MIX of AND - OR****')
        results,comparisons = process_MIX(query, temp_results, ind)
        
    print('#####Total Comparisons : {}#####'.format(comparisons))
    return results
def print_results(results):
    docs = len(results)
    print('Total Docs Retrieved are : {}'.format(docs))
    files = []
    for id in results:
        files.append(file_mapper[id])
    return files   
    
def read_query():
    query = input().split()
    results = process_query(query)
    files = print_results(results)
    return files

In [57]:
files = read_query()


email and boy or not girl
***final query after preprocessing***
['email', 'and', 'boy', 'or', 'not', 'girl']
***Processing NOT Operator***
['email', 'and', 'boy', 'or', 'result0']
**** MIX of AND - OR****
['result1', 'or', 'result0']
#####Total Comparisons : 22365#####
Total Docs Retrieved are : 19841
